# Tratamento de Dados
  * Limpeza de Dados: Remoção de entradas duplicadas ou irrelevantes.
  * Normalização: Escalonamento dos dados para que todas as variáveis tenham a mesma importância. Isso pode ser feito usando técnicas como Min-Max Scaling ou Z-score normalization.
  * Transformação de Variáveis Categóricas: Conversão de variáveis categóricas em numéricas, utilizando técnicas como one-hot encoding.
  * Redução de Dimensionalidade: Aplicação de técnicas como PCA (Análise de Componentes Principais) para reduzir o número de características, mantendo a maior quantidade de informação possível.

In [1]:
# importar bibliotecas necessárias
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report,
    roc_auc_score, roc_curve
    )
from sklearn.neighbors import KNeighborsClassifier
import gc

In [2]:
# Carregar o arquivo (sem cabeçalhos)
# Carregar o arquivo (sem cabeçalhos)
colunas = pd.read_csv("NUSW-NB15_features.csv", encoding='latin-1')
colunas["Name"] = colunas["Name"].str.replace(' ', '_')
colunas = colunas["Name"].tolist()
colunas

['srcip',
 'sport',
 'dstip',
 'dsport',
 'proto',
 'state',
 'dur',
 'sbytes',
 'dbytes',
 'sttl',
 'dttl',
 'sloss',
 'dloss',
 'service',
 'Sload',
 'Dload',
 'Spkts',
 'Dpkts',
 'swin',
 'dwin',
 'stcpb',
 'dtcpb',
 'smeansz',
 'dmeansz',
 'trans_depth',
 'res_bdy_len',
 'Sjit',
 'Djit',
 'Stime',
 'Ltime',
 'Sintpkt',
 'Dintpkt',
 'tcprtt',
 'synack',
 'ackdat',
 'is_sm_ips_ports',
 'ct_state_ttl',
 'ct_flw_http_mthd',
 'is_ftp_login',
 'ct_ftp_cmd',
 'ct_srv_src',
 'ct_srv_dst',
 'ct_dst_ltm',
 'ct_src__ltm',
 'ct_src_dport_ltm',
 'ct_dst_sport_ltm',
 'ct_dst_src_ltm',
 'attack_cat',
 'Label']

In [ ]:
csv_files = ['UNSW-NB15_1.csv', 'UNSW-NB15_2.csv',
             'UNSW-NB15_3.csv', 'UNSW-NB15_4.csv']

# Create an empty dataframe to store the combined data
dados = pd.DataFrame()

# Loop through each CSV file and append its contents
#to the combined dataframe
for csv_file in csv_files:
    df = pd.read_csv(csv_file, header=None, names=colunas)
    dados = pd.concat([dados, df])

<ipython-input-3-a8aa636aaaa7>:10: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file, header=None, names=colunas)


In [ ]:
csv_files[1:3]

In [ ]:
# prompt: na tabela substitua as ocorecians de nan na coluna attack_cat por normal

dados['attack_cat'] = dados['attack_cat'].fillna('Normal')
dados['attack_cat'] = dados['attack_cat'].replace('Backdoors', "Backdoor")
dados['attack_cat'] = dados['attack_cat'].replace(' Fuzzers ', " Fuzzers")
dados['ct_flw_http_mthd'] = dados['ct_flw_http_mthd'].fillna(0)
dados['is_ftp_login'] = dados['is_ftp_login'].fillna(0)
dados['ct_ftp_cmd'] = dados['ct_ftp_cmd'].replace(' ', 0)

dados['service'] = dados['service'].replace('-', 'none')

In [ ]:
dados.head()

In [ ]:
dados.shape

In [ ]:
dados.drop(columns=["srcip", "dstip", "Label"], inplace=True)

In [ ]:
dados.shape

(2540047, 46)

1. # Tratamento de Valores Ausentes

In [ ]:
#1. Limpeza de Dados (Remoção de Entradas Duplicadas ou Irrelevantes)
# Remover entradas duplicadas
orig_dados = dados
oring_label = dados["attack_cat"]
dados = dados.drop_duplicates()
# Remover linhas com valores nulos
dados = dados.dropna()
dados.reset_index(drop=True, inplace=True)

In [ ]:
dados.shape

(2058994, 46)

2. # Detecção e Tratamento de Outliers
com z-score

In [ ]:
#features que podem conter outlines
features_with_outlines = ["dur", "sbytes" , "dbytes", "Sload" ,
 "Dload", "Spkts" , "Dpkts", "swin" , "dwin", "smeansz" , "dmeansz", "trans_depth",
"tcprtt", "synack"]

ct_state_ttl: 0.1223
dttl: 0.1203
dwin: 0.1065
sttl: 0.0852
ct_flw_http_mthd: 0.0698
service: 0.0662
trans_depth: 0.0591

In [ ]:
# Função para remoção de outliers com Z-Score
def remove_outliers_zscore(df, column, threshold=4):
    mean = df[column].mean()
    std_dev = df[column].std()
    z_scores = (df[column] - mean) / std_dev
    return df[np.abs(z_scores) < threshold]

''' for feature in features_with_outlines:
    dados = remove_outliers_zscore(dados, feature, threshold=3) '''
def apply_outlier_removal_conditional(df, features_with_outlines, min_class_size=3000):
    # Separar o DataFrame pelas classes
    class_groups = df.groupby('attack_cat')

    df_cleaned = pd.DataFrame()  # DataFrame para armazenar os dados limpos

    for class_label, group in class_groups:
        if len(group) > min_class_size:  # Se a classe tiver mais de 5000 amostras
            for feature in features_with_outlines:
                if class_label == "Normal":
                  group = remove_outliers_zscore(group, feature, threshold=1)
                  #print(class_label, len(group))
                else: group = remove_outliers_zscore(group, feature)
                #print(class_label, len(group))
        # Adicionar os dados limpos ao DataFrame final
        df_cleaned = pd.concat([df_cleaned, group])

    return df_cleaned

# Aplicar a remoção de outliers
dados = apply_outlier_removal_conditional(dados, features_with_outlines)

In [ ]:
dados.shape

(336202, 46)

## Pegando o Label

In [ ]:
# Create a LabelEncoder object
label_encoder = LabelEncoder()
# Fit the encoder on the 'attack' column and transform it
dados['attack_cat'] = label_encoder.fit_transform(dados['attack_cat'])
label = dados['attack_cat']
dados.drop(columns=["attack_cat"], inplace=True)


In [ ]:
label.value_counts()#979689

,count
attack_cat,
8,245965
6,25332
7,21718
0,19463
1,11137
5,4996
3,2185
4,1984
9,1740


###Separação das Variáveis Numéricas e Categoricas

In [ ]:
# Separar os dados numéricos e categóricos
dados.reset_index(drop=True)
dados_numericos = dados.select_dtypes(include=["int64", "float64"])
dados_categoricos = dados.select_dtypes(include=["object"])

In [ ]:
# Resetar os índices antes de concatenar
dados_numericos.reset_index(drop=True, inplace=True)
dados_categoricos.reset_index(drop=True, inplace=True)
label.reset_index(drop=True, inplace=True)
dados_numericos.shape, dados_categoricos.shape, label.shape

((336202, 39), (336202, 6), (336202,))

3. # Transformação e `Normalização`


In [ ]:
# 3. Normalizar dados numéricos
scaler = MinMaxScaler()
dados_numericos_normalizados = pd.DataFrame(scaler.fit_transform(dados_numericos), columns=dados_numericos.columns)

4. # Codificação de Variáveis Categóricas

In [ ]:
for col in dados_categoricos.columns:
    dados_categoricos[col] = dados_categoricos[col].astype(str)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Aplicar Label Encoding em cada coluna categórica
for col in dados_categoricos.columns:
    label_encoder1 = LabelEncoder()
    dados_categoricos[col] = label_encoder1.fit_transform(dados_categoricos[col])


In [ ]:
dados_numericos_normalizados.shape, dados_categoricos.shape, label.shape

((336202, 39), (336202, 6), (336202,))

### Recombinar os dados

In [ ]:
#Combinar tudo
dados_normalizados = pd.concat([dados_numericos_normalizados, dados_categoricos, label], axis=1)

In [ ]:
dados_normalizados

,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,Sload,Dload,Spkts,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,sport,dsport,proto,state,service,ct_ftp_cmd,attack_cat
0,0.008644,0.004088,0.000000,0.996078,0.000000,0.000000,0.000000,0.000005,0.000000,0.076305,...,0.015152,0.027027,0.015152,0,0,75,4,6,0,0
1,0.020313,0.012071,0.000000,0.996078,0.000000,0.000000,0.000000,0.000006,0.000000,0.108434,...,0.015152,0.027027,0.015152,0,0,75,4,6,0,0
2,0.003010,0.002438,0.000293,0.996078,0.996047,0.028986,0.002882,0.000008,0.001259,0.036145,...,0.030303,0.000000,0.030303,47152,31286,109,3,4,0,0
3,0.003692,0.002358,0.000293,0.996078,0.996047,0.028986,0.002882,0.000007,0.001027,0.036145,...,0.015152,0.000000,0.015152,25075,4472,109,3,6,0,0
4,0.004377,0.002358,0.000293,0.996078,0.996047,0.028986,0.002882,0.000006,0.000866,0.036145,...,0.015152,0.000000,0.015152,60579,4472,109,3,6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336197,0.029718,0.007975,0.288121,0.996078,0.996047,0.028986,0.288184,0.000003,0.149446,0.148594,...,0.000000,0.000000,0.000000,22169,31286,109,3,4,0,11
336198,0.011594,0.005110,0.000293,0.996078,0.996047,0.028986,0.002882,0.000005,0.000327,0.036145,...,0.000000,0.000000,0.000000,24031,31286,109,3,4,0,11
336199,0.003428,0.005190,0.000293,0.996078,0.996047,0.028986,0.002882,0.000015,0.001106,0.036145,...,0.000000,0.000000,0.000000,40199,31286,109,3,4,0,11
336200,0.018236,0.005166,0.000387,0.996078,0.996047,0.028986,0.002882,0.000003,0.000288,0.036145,...,0.000000,0.000000,0.000000,1499,31286,109,3,4,0,11


In [ ]:
# prompt: drop todo dado de que na coluna attack_cat tenha normal

# Supondo que 'dados_normalizados' seja o seu DataFrame
# e que 'attack_cat' seja a coluna que você quer filtrar

dados_normalizados_filtrados = dados_normalizados[dados_normalizados['attack_cat'] != 8] # Substitua 0 pelo valor correspondente a normal

# Agora 'dados_normalizados_filtrados' contém apenas as linhas onde attack_cat é "normal".
# Você pode realizar as operações desejadas com este novo dataframe.
dados_normalizados = dados_normalizados_filtrados

### Separar dados e label

In [ ]:
X = dados_normalizados.drop(columns=["attack_cat"])
y = dados_normalizados["attack_cat"]

In [ ]:
y.value_counts(), y.shape

(attack_cat
 6     25332
 7     21718
 0     19463
 1     11137
 5      4996
 3      2185
 4      1984
 9      1740
 2      1288
 10      223
 11      171
 Name: count, dtype: int64,
 (90237,))

# Divisão do dataset
* ### 60%, 40%

In [ ]:
# Divida o conjunto inicial de dados em treino (60%) e o restante (40%) para validação e teste
X_treino, X_validacao, y_treino, y_validacao = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

# Verifique os tamanhos
print("Tamanho do conjunto de treino:", len(X_treino), " | ",(len(X)/100)*60)
print("Tamanho do conjunto de validação:", len(X_validacao), " | ",(len(X)/100)*20)

Tamanho do conjunto de treino: 54142  |  54142.2
Tamanho do conjunto de validação: 36095  |  18047.4


### Melhorando o treinamento

In [ ]:
!pip install scikit-learn==1.2.0

In [ ]:
''' from imblearn.over_sampling import SMOTE
from collections import Counter

# ... (your existing code) ...

# Obter a distribuição atual das classes
classe_distribuicao = Counter(y_treino)
print(f"Distribuição das classes antes: {classe_distribuicao}")

# Definir a proporção desejada para cada classe em relação à majoritária
# Por exemplo, aumentar todas as classes para o tamanho da classe majoritária
majoritaria = max(classe_distribuicao.values())
sampling_strategy ={0: 19463,
         1: 11137,
         2: int(1288*1.2),
         3: int(2185*1.1),
         4: int(1984*1.2),
         5: 4996,
         6: 25332,
         7: 21718,
         8: 913505,
         9: int(1740*1.2),
         10: 223*6,
         11: 171*10
}
#  ^-- Change here: Only oversample minority classes

# Aplicar o SMOTE
smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)
X_treino, y_treino = smote.fit_resample(X_treino, y_treino)
 {8.0: 913505, 6.0: 13812, 7.0: 12532, 0.0: 10016, 1.0: 7127, 5.0: 2753,
3.0: 1193, 4.0: 1010, 2.0: 819, 9.0: 93}
# ... (rest of your code) ... '''

' from imblearn.over_sampling import SMOTE\nfrom collections import Counter\n\n# ... (your existing code) ...\n\n# Obter a distribuição atual das classes\nclasse_distribuicao = Counter(y_treino)\nprint(f"Distribuição das classes antes: {classe_distribuicao}")\n\n# Definir a proporção desejada para cada classe em relação à majoritária\n# Por exemplo, aumentar todas as classes para o tamanho da classe majoritária\nmajoritaria = max(classe_distribuicao.values())\nsampling_strategy ={0: 19463,\n         1: 11137,\n         2: int(1288*1.2),\n         3: int(2185*1.1),\n         4: int(1984*1.2),\n         5: 4996,\n         6: 25332,\n         7: 21718,\n         8: 913505,\n         9: int(1740*1.2),\n         10: 223*6,\n         11: 171*10\n}\n#  ^-- Change here: Only oversample minority classes\n\n# Aplicar o SMOTE\nsmote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)\nX_treino, y_treino = smote.fit_resample(X_treino, y_treino)\n {8.0: 913505, 6.0: 13812, 7.0: 12532, 0.0

### Criando um relatorio da quantidade das classes em cada tratamento e divisão

In [ ]:
import pandas as pd
classes_originais = label_encoder.classes_
# Suponha que y, y_treino, y_validacao e y_teste são Series pandas
# (já com os valores de classe contados anteriormente)

# Contando as classes em cada conjunto
contagem_y = y.value_counts()
contagem_treino = y_treino.value_counts()
contagem_validacao = y_validacao.value_counts()
#contagem_teste = y_teste.value_counts()

# Criando um DataFrame com as contagens
df_comparacao = pd.DataFrame({
    'Classe': contagem_y.index,  # Pegando as classes
    'Classe_Original': [classes_originais[int(i)] for i in contagem_y.index],
    "Sem tratamento": oring_label.value_counts(),
    'com tratamento': contagem_y.values,
    'Treino': contagem_treino.reindex(contagem_y.index, fill_value=0).values,
    'Validacao': contagem_validacao.reindex(contagem_y.index, fill_value=0).values
})

ValueError: array length 11 does not match index length 12

In [ ]:
len(y.unique()),len(y_treino.unique()), len(y_validacao.unique())

In [ ]:
df_comparacao

# Treinamento do modelo
* ### XGBoost - Classificação
#### Gradient Boosting Decision Trees
Encadeamento de Arveres de decisões em que o resultado da arvere influencia nos pesos da proxima arvere.

## Parâmetros:

### Objetivo para multiclasse
- objective='multi:softmax' ou 'multi:softprob':

 - 'multi:softmax': Retorna a classe diretamente.
 - 'multi:softprob': Retorna probabilidades para cada classe (caso
  queira análises adicionais).

- num_class=23:
  - Corresponde ao número de classes no seu dataset.

### Principais
- learning_rate:
   - Taxa de aprendizado. Pode começar com 0.1. Use menor (0.01) se aumentar o número de estimadores (n_estimators).
- n_estimators:

 - Quantidade de árvores. Um valor inicial é 200. Para mais precisão, aumente para 500, mas isso exigirá mais memória e tempo.

- max_depth:

 - Profundidade das árvores. Comece com 6. Ajuste para 4-8 dependendo do overfitting.
- subsample e colsample_bytree:

 - Controlam a fração de amostras e features usadas em cada árvore, respectivamente. Esses valores ajudam a prevenir overfitting.

### Desempenho
- tree_method='hist':

 - Método de construção das árvores otimizado para grandes datasets no Colab.
- reg_alpha e reg_lambda:

 - Regularização para controlar overfitting. Valores iniciais razoáveis são 0.1 e 1.

In [ ]:
params = {
    "um": {
      "objective": "multi:softmax",  # Classificação multiclasse
      "num_class": len(set(y)),  # Número de classes
      "max_depth": 6,  # Profundidade máxima da árvore
      "eta": 0.1,  # Taxa de aprendizado
      "subsample": 0.8,  # Amostragem de linhas
      "colsample_bytree": 0.8,  # Amostragem de colunas
      "scale_pos_weight": 1,  # Ajustar com base no desbalanceamento
      "eval_metric": "mlogloss",  # Métrica de avaliação
    }
}

In [ ]:
!pip install scikit-learn==1.2.0

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier( objective='multi:softmax',
                      eval_metric='mlogloss',
                      num_class=len(y.unique()),
                      learning_rate=0.1,
                      n_estimators=200,
                       max_depth=6,
                       min_child_weight=3,
                       gamma=0.1,
                       subsample=0.8,
                       colsample_bytree=0.8,
                       scale_pos_weight=1,
                       reg_alpha=0.1,
                       reg_lambda=1,
                       tree_method='hist',
                       random_state=42)



In [ ]:
from sklearn.utils.class_weight import compute_sample_weight
sample_weights = compute_sample_weight(class_weight='balanced', y=y_treino)
model.fit(X_treino, y_treino, sample_weight=sample_weights)

In [ ]:
# Importância das features
importances = model.feature_importances_
features = X.columns

# Mostrar as 10 features mais importantes
feature_importance = sorted(zip(features, importances), key=lambda x: x[1], reverse=True)
for feature, importance in feature_importance[:10]:
    print(f"{feature}: {importance:.4f}")

In [ ]:
y_pred = model.predict(X_treino)
y_pred_proba = model.predict_proba(X_treino)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score, roc_curve

In [ ]:
matriz_confusao = confusion_matrix(y_treino, y_pred)

In [ ]:
# Calcular a acurácia do modelo
report = classification_report(y_treino, y_pred, output_dict=True)
accuracy = accuracy_score(y_treino, y_pred)
precisao = precision_score(y_treino, y_pred, average='weighted')
recall = recall_score(y_treino, y_pred, average='weighted')
f1 = f1_score(y_treino, y_pred, average='weighted')

In [ ]:
overall_accuracy = report['accuracy']
print(f"\nOverall Accuracy (from classification report): {overall_accuracy * 100:.2f}%")
# ... (rest of your existing code)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# ... (your existing code) ...

# Calculate the confusion matrix
# Plot the confusion matrix using seaborn
plt.figure(figsize=(11, 9))
sns.heatmap(matriz_confusao,  annot=True, fmt="d", cmap="YlGnBu", linewidths=0.5, xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix treino")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report

# Exemplo: Gerar o classification_report (substitua pelo seu y_validacao e y_pred_v)
# report_v = classification_report(y_validacao, y_pred_v, output_dict=True)

# 1. Transformar o relatório em DataFrame
df_report = pd.DataFrame(report).T

# 2. Separar as métricas das classes e médias gerais
classes = df_report.iloc[:-3, :3]  # Todas as classes (exclui últimas linhas como 'accuracy')
medias = df_report.iloc[-2:, :3]  # 'macro avg' e 'weighted avg'

# 3. Gráfico 1: Métricas por Classe
plt.figure(figsize=(10, 5))
classes.plot(kind='bar', figsize=(12, 6), colormap='viridis')
plt.title("Métricas por Classe (Precision, Recall, F1-Score)", fontsize=16)
plt.xlabel("Classes", fontsize=12)
plt.ylabel("Valor", fontsize=12)
plt.xticks(rotation=0)
plt.legend(title="Métricas", loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
y_pred_v = model.predict(X_validacao)
y_pred_proba_v = model.predict_proba(X_validacao)

In [ ]:
matriz_confusao_v = confusion_matrix(y_validacao, y_pred_v)

In [ ]:
len(y_validacao), len(y_pred_v)

In [ ]:
# Calcular a acurácia do modelo
report_v = classification_report(y_validacao, y_pred_v, output_dict=True)
accuracy_v = report_v['accuracy']
precisao_v = report_v['weighted avg']['precision']
recall_v = report_v['weighted avg']['recall']
f1_v = report_v['weighted avg']['f1-score']

In [ ]:

overall_accuracy = report_v['accuracy']
print(f"\nOverall Accuracy (from classification report): {overall_accuracy * 100:.2f}%")
# ... (rest of your existing code)

In [ ]:
# prompt: gere uma matriz de confusão para y_validacao e y_pred_v

# ... (your existing code) ...

# Calculate the confusion matrix for validation set
matriz_confusao_v = confusion_matrix(y_validacao, y_pred_v)

# Plot the confusion matrix for validation set using seaborn
plt.figure(figsize=(15, 10))
sns.heatmap(matriz_confusao_v, annot=True, fmt="d", cmap="YlGnBu", linewidths=0.5, xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix for Validation Set")
plt.show()

In [ ]:
arr = y_pred_v
# Contar as ocorrências de 9
ocorrencias_9 = np.count_nonzero(arr == 17)
ocorrencias_9, #0 #2008

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report

# Exemplo: Gerar o classification_report (substitua pelo seu y_validacao e y_pred_v)
# report_v = classification_report(y_validacao, y_pred_v, output_dict=True)

# 1. Transformar o relatório em DataFrame
df_report = pd.DataFrame(report_v).T

# 2. Separar as métricas das classes e médias gerais
classes = df_report.iloc[:-3, :3]  # Todas as classes (exclui últimas linhas como 'accuracy')

# 3. Gráfico 1: Métricas por Classe
plt.figure(figsize=(12, 6))
classes.plot(kind='bar', figsize=(12, 6), colormap='viridis')
plt.title("Métricas por Classe (Precision, Recall, F1-Score)", fontsize=16)
plt.xlabel("Classes", fontsize=12)
plt.ylabel("Valor", fontsize=12)
plt.xticks(rotation=0)
plt.xticks(ticks=range(len(label_encoder.classes_)), labels=label_encoder.classes_, rotation=0, ha="center")
plt.legend(title="Métricas", loc='upper right')
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc
# Converter os rótulos de y_test para binário
y_test_binary = (y_validacao == 'normal.').astype(int)

# 10. Calcular a Curva ROC e AUC
y_pred_prob = y_pred_proba_v[:, 1]  # Probabilidades de classe positiva
fpr, tpr, thresholds = roc_curve(y_test_binary, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plotar a Curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='Curva ROC (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.title('Curva ROC e Área Sob a Curva (AUC)')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# prompt: compare os resultados da precision das classes meu treino de da minha validação, plot em um grafico das barras.

import matplotlib.pyplot as plt

# Dados de precisão (substitua pelos seus valores reais)
classes = list(range(len(y.unique())))  # Assumindo 23 classes
precision_treino = [report[str(i)]['precision'] for i in classes]
precision_validacao = [report_v[str(i)]['precision'] for i in classes]


# Criar gráfico de barras
x = np.arange(len(classes))  # Posições das barras no eixo x
width = 0.35  # Largura das barras

fig, ax = plt.subplots(figsize=(15, 6))
rects1 = ax.bar(x - width/2, precision_treino, width, label='Treino')
rects2 = ax.bar(x + width/2, precision_validacao, width, label='Validação')

# Adicionar rótulos e título
ax.set_ylabel('Precisão')
ax.set_title('Comparação da Precisão entre Treino e Validação')
ax.set_xticks(x)
ax.set_xticklabels(classes)
ax.legend()

# Adicionar valores acima das barras
def autolabel(rects):
    """Adiciona rótulos acima das barras."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{:.2f}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)

fig.tight_layout()
plt.show()

In [ ]:
# prompt: gere um codigo que compare as metricas da minha validação e do meu treino fq, recall, accuracy e recision. Coloque os valores de cada um

import pandas as pd
import matplotlib.pyplot as plt
from copy import copy

# Assuming 'report' and 'report_v' are already defined from your previous code
# and contain the classification reports for training and validation sets.

def compare_metrics(report, report_v):
    """Compares metrics (precision, recall, accuracy, f1-score) between training and validation sets."""

    metrics_train = copy(report['weighted avg'])
    metrics_train['accuracy'] = report['accuracy']
    metrics_train.pop('support')

    metrics_val = copy(report_v['weighted avg'])
    metrics_val['accuracy'] = report_v['accuracy']
    metrics_val.pop('support')

    metric_names = list(metrics_train.keys())

    # Create a DataFrame for better visualization
    df_comparison = pd.DataFrame({
        'Metric': metric_names,
        'Train': list(metrics_train.values()),
        'Validation': list(metrics_val.values())
    })

    #print(df_comparison)

    # Create bar plot
    plt.figure(figsize=(10, 6))
    X_axis = np.arange(len(metric_names))

    plt.bar(X_axis - 0.2, metrics_train.values(), 0.4, label='Training')
    plt.bar(X_axis + 0.2, metrics_val.values(), 0.4, label='Validation')

    plt.xticks(X_axis, metric_names)
    plt.xlabel("Metrics")
    plt.ylabel("Value")
    plt.title("Comparison of Metrics")
    plt.legend()

    # Add values on top of bars
    for i, v in enumerate(metrics_train.values()):
      plt.text(i-0.2, v + 0.01, f"{v:.4f}", ha='center')
    for i, v in enumerate(metrics_val.values()):
      plt.text(i+0.2, v + 0.01, f"{v:.4f}", ha='center')
    plt.show()


compare_metrics(report, report_v)